In [1]:
import os
import sys
import numpy as np
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import cv2

import matplotlib.pyplot as plt
plt.switch_backend('agg')  # allows code to run without a system DISPLAY


Using plaidml.keras.backend backend.


In [2]:
class GAN():
    def __init__(self, width=32, height=32, channels=1):
        self.width = width
        self.height = height
        self.channels = channels

        self.shape = (self.width, self.height, self.channels)

        self.optimizer = Adam(lr=0.0002, beta_1=0.5, decay=8e-8)

        self.G = self.__generator()
        self.G.compile(loss='binary_crossentropy', optimizer=self.optimizer)

        self.D = self.__discriminator()
        self.D.compile(
            loss='binary_crossentropy',
            optimizer=self.optimizer,
            metrics=['accuracy'])

        self.stacked_generator_discriminator = self.__stacked_generator_discriminator(
        )
        self.stacked_generator_discriminator.compile(
            loss='binary_crossentropy', optimizer=self.optimizer)

    def __generator(self):
        """ Declare generator """
        model = Sequential()
        model.add(Dense(256, input_shape=(100, )))
        model.add(LeakyReLU(
            alpha=0.2))  # Using leaky relu as internal activation function
        model.add(BatchNormalization(
            momentum=0.8))  # Optimize with BatchNormalization
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(
            Dense(self.width * self.height * self.channels, activation='tanh')
        )  # Using tanh as the final layer's activation function
        model.add(Reshape((self.width, self.height, self.channels)))
        model.summary()

        return model

    def __discriminator(self):
        """ Declare discriminator """
        model = Sequential()
        model.add(Flatten(input_shape=self.shape))
        model.add(
            Dense((self.width * self.height * self.channels),
                  input_shape=self.shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(int((self.width * self.height * self.channels) / 2)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        return model

    def __stacked_generator_discriminator(self):
        self.D.trainable = False

        model = Sequential()
        model.add(self.G)
        model.add(self.D)

        return model

    def train(self, X_train, epochs=10000, batch=32, save_interval=100):
        for cnt in range(epochs + 1):
            ## train discriminator
            random_index = np.random.randint(0, len(X_train) - batch / 2)
            legit_images = X_train[random_index:random_index +
                                   int(batch / 2)].reshape(
                                       int(batch / 2), self.width, self.height,
                                       self.channels)

            gen_noise = np.random.normal(0, 1, (int(batch / 2), 100))
            syntetic_images = self.G.predict(gen_noise)

            x_combined_batch = np.concatenate((legit_images, syntetic_images))
            y_combined_batch = np.concatenate((np.ones((int(batch / 2), 1)),
                                               np.zeros((int(batch / 2), 1))))

            d_loss = self.D.train_on_batch(x_combined_batch, y_combined_batch)

            # train generator

            noise = np.random.normal(0, 1, (batch, 100))  # add Gaussian noises
            y_mislabled = np.ones((batch, 1))

            g_loss = self.stacked_generator_discriminator.train_on_batch(
                noise, y_mislabled)

            print(
                'epoch: %d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f]'
                % (cnt, d_loss[0], g_loss))

            if cnt % save_interval == 0:
                self.plot_images(save2file=True, step=cnt)

    def plot_images(self, save2file=False, samples=16, step=0):
        ''' Plot and generated images '''
        filename = "./images/bike_%d.png" % step
        noise = np.random.normal(0, 1, (samples, 100))

        images = self.G.predict(noise)

        plt.figure(figsize=(10, 10))

        for i in range(images.shape[0]):
            plt.subplot(4, 4, i + 1)
            image = images[i, :, :, :]
            image = np.reshape(image, [self.height, self.width])
            plt.imshow(image, cmap='gray')
            plt.axis('off')
        plt.tight_layout()

        if save2file:
            plt.savefig(filename)
            plt.close('all')
        else:
            plt.show()

In [3]:
saved_img = "./images"
if not os.path.exists(saved_img):
    os.makedirs(saved_img)
files = ["./data/train/bike/" + f for f in os.listdir("./data/train/bike")]
X = []
y = []
for img in files:
    data = cv2.imread(img, cv2.IMREAD_GRAYSCALE) / 255.0
    data = cv2.resize(data, (32, 32))
    label = 0
    X.append(data)
    y.append(label)
X_train, _, _, _ = train_test_split(X, y, test_size=0.2)

In [4]:
X_train = np.array(X_train)
gan = GAN()
gan.train(X_train)

INFO:plaidml:Opening device "opencl_amd_ellesmere.0"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_2 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              525312    
__________

D:\CS\Anaconda\lib\site-packages\keras\engine\training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch: 0, [Discriminator :: d_loss: 0.511724], [ Generator :: loss: 0.590783]
epoch: 1, [Discriminator :: d_loss: 0.380976], [ Generator :: loss: 0.700087]
epoch: 2, [Discriminator :: d_loss: 0.350381], [ Generator :: loss: 0.803713]
epoch: 3, [Discriminator :: d_loss: 0.289390], [ Generator :: loss: 1.079037]
epoch: 4, [Discriminator :: d_loss: 0.277482], [ Generator :: loss: 1.404783]
epoch: 5, [Discriminator :: d_loss: 0.215204], [ Generator :: loss: 1.925699]
epoch: 6, [Discriminator :: d_loss: 0.147955], [ Generator :: loss: 2.332766]
epoch: 7, [Discriminator :: d_loss: 0.114482], [ Generator :: loss: 2.833924]
epoch: 8, [Discriminator :: d_loss: 0.056039], [ Generator :: loss: 3.129716]
epoch: 9, [Discriminator :: d_loss: 0.051504], [ Generator :: loss: 3.277596]
epoch: 10, [Discriminator :: d_loss: 0.032148], [ Generator :: loss: 3.330954]
epoch: 11, [Discriminator :: d_loss: 0.037863], [ Generator :: loss: 3.658172]
epoch: 12, [Discriminator :: d_loss: 0.028895], [ Generator ::

epoch: 106, [Discriminator :: d_loss: 0.047691], [ Generator :: loss: 5.443333]
epoch: 107, [Discriminator :: d_loss: 0.044183], [ Generator :: loss: 5.268557]
epoch: 108, [Discriminator :: d_loss: 0.093047], [ Generator :: loss: 5.286050]
epoch: 109, [Discriminator :: d_loss: 0.026212], [ Generator :: loss: 5.301665]
epoch: 110, [Discriminator :: d_loss: 0.018043], [ Generator :: loss: 5.178346]
epoch: 111, [Discriminator :: d_loss: 0.077666], [ Generator :: loss: 5.211918]
epoch: 112, [Discriminator :: d_loss: 0.119742], [ Generator :: loss: 5.370410]
epoch: 113, [Discriminator :: d_loss: 0.035637], [ Generator :: loss: 5.154692]
epoch: 114, [Discriminator :: d_loss: 0.123328], [ Generator :: loss: 5.188578]
epoch: 115, [Discriminator :: d_loss: 0.019465], [ Generator :: loss: 4.850069]
epoch: 116, [Discriminator :: d_loss: 0.099290], [ Generator :: loss: 5.686638]
epoch: 117, [Discriminator :: d_loss: 0.068323], [ Generator :: loss: 5.106537]
epoch: 118, [Discriminator :: d_loss: 0.

KeyboardInterrupt: 